In [1]:
%load_ext autoreload
%autoreload 2
%load_ext tensorboard

In [2]:
from random import random

from SourceCodeTools.models.training_config import get_config, save_config, load_config
from SourceCodeTools.code.data.dataset.Dataset import SourceGraphDataset, filter_dst_by_freq
from SourceCodeTools.models.graph.train.sampling_multitask2 import training_procedure, SamplingMultitaskTrainer
from SourceCodeTools.models.graph.train.objectives.NodeClassificationObjective import NodeClassifierObjective
from SourceCodeTools.models.graph.train.objectives.SubgraphClassifierObjective import SubgraphAbstractObjective, \
    SubgraphClassifierObjective, SubgraphEmbeddingObjective
from SourceCodeTools.models.graph.train.utils import get_name, get_model_base
from SourceCodeTools.models.graph import RGGAN
from SourceCodeTools.tabular.common import compact_property
from SourceCodeTools.code.data.file_utils import unpersist
from SourceCodeTools.models.graph.train.objectives.SCAAClassifierObjetive import SCAAClassifierObjective
import dgl
import torch
import numpy as np
from argparse import Namespace
from torch import nn
from datetime import datetime
from os.path import join
from functools import partial

# Prepare parameters and options

Full list of options that can be added can be found in `SourceCodeTools/models/training_options.py`. They are ment to be used as arguments for cli trainer. Trainer script can be found in `SourceCodeTools/scripts/train.py`.

For the task of subgraph classification the important options are:
- `subgraph_partition` is path to subgraph-based train/val/test sets. Storead as Dataframe with subgraph id and partition mask
- `subgraph_id_column` is a column is `common_edges` file that stores subgraph id.
- For variable misuse task (same will apply to authorship attribution) subgraphs are created for individual functions (files for SCAA). The label is stored in `common_filecontent`.

In [3]:
tokenizer_path = "examples/sentencepiece_bpe.model"

data_path = "examples/one_to_one/with_ast"
subgraph_partition = join(data_path, "subgraph_partition.json.bz2")
filecontent_path = join(data_path, "common_filecontent.json.bz2")

In [4]:
unpersist(filecontent_path)

,Unnamed: 0.1,Unnamed: 0,task,user,year,package,filecontent,id
0,2498,2498,3,alex1770,2020,False,\ndef getstring(): return input().strip()\ndef...,2498
1,4644,4644,4,sashadotcom,2020,False,import sys\nfrom itertools import combinations...,4644
2,4864,4864,4,superkjelle,2020,False,"[t,b] = input().split(' ')\nt = int(t)\nb = in...",4864
3,270,270,0,BananaNosh,2020,False,"PARENTS = {0: ""C"", 1: ""J""}\n\n\ndef print_outp...",270
4,5400,205,7,TinyTitans,2020,False,def TestCase():\n N = int(input())\n ans...,5400
...,...,...,...,...,...,...,...,...
5533,289,289,4,Basti_SUI,2020,False,import sys\n\n\ndef inverse(arr):\n return ...,289
5534,2556,2556,1,anari,2020,False,T = int(input())\nfor t in range(T):\n N = ...,2556
5535,315,315,0,Bhavyalekha,2020,False,def f(raw_arr):\n arr = []\n for i in ra...,315
5536,1531,1531,1,Poornachander,2020,False,t1=int(input())\nli=[]\nfor j in range(len(li)...,1531


In [5]:
unpersist(join(data_path, "common_edges.json.bz2"), nrows=10)

,id,type,source_node_id,target_node_id,file_id,mentioned_in
0,0,subword,213905,516482,2498,NaN
1,1,func,516482,257761,2498,46283.0
2,2,func_rev,257761,516482,2498,46283.0
3,3,value,257761,107985,2498,46283.0
4,4,value_rev,107985,257761,2498,46283.0
...,...,...,...,...,...,...
285,2961294,prev,208030,698551,5397,693983.0
286,2961295,defined_in_module,208030,693983,5397,693983.0
287,2961296,defined_in_module_rev,693983,208030,5397,693983.0
288,2961297,subword,708231,433611,5397,NaN


In [6]:
config = get_config(
    subgraph_id_column="file_id",
    data_path=data_path,
    model_output_dir=data_path,
    subgraph_partition=subgraph_partition,
    tokenizer_path=tokenizer_path,
    objectives="subgraph_clf",
    epochs=1000,
    elem_emb_size=6, # the smallest size of the graph in dataset
)

In [7]:
save_config(config, "var_misuse_tiny.yaml")

In [8]:
config

{'DATASET': {'data_path': 'examples/one_to_one/with_ast',
  'train_frac': 0.9,
  'filter_edges': None,
  'min_count_for_objectives': 5,
  'self_loops': False,
  'use_node_types': False,
  'use_edge_types': False,
  'no_global_edges': False,
  'remove_reverse': False,
  'custom_reverse': None,
  'restricted_id_pool': None,
  'random_seed': None,
  'subgraph_id_column': 'file_id',
  'subgraph_partition': 'examples/one_to_one/with_ast/subgraph_partition.json.bz2'},
 'TRAINING': {'model_output_dir': 'examples/one_to_one/with_ast',
  'pretrained': None,
  'pretraining_phase': 0,
  'sampling_neighbourhood_size': 10,
  'neg_sampling_factor': 3,
  'use_layer_scheduling': False,
  'schedule_layers_every': 10,
  'elem_emb_size': 6,
  'embedding_table_size': 200000,
  'epochs': 1000,
  'batch_size': 128,
  'learning_rate': 0.001,
  'objectives': 'subgraph_clf',
  'save_each_epoch': False,
  'save_checkpoints': True,
  'early_stopping': False,
  'early_stopping_tolerance': 20,
  'force_w2v_ns': Fa

# Create Dataset

In [9]:
dataset = SourceGraphDataset(
    **{**config["DATASET"], **config["TOKENIZER"]}
)

/home/alina/workspace/scaa/method-embedding/SourceCodeTools/code/data/dataset/Dataset.py:129: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.nodes = self.nodes.append(new_nodes, ignore_index=True)
/home/alina/workspace/scaa/method-embedding/SourceCodeTools/code/data/dataset/Dataset.py:130: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.edges = self.edges.append(new_edges, ignore_index=True)


# Declare target loading function (labels)

In [10]:
def load_labels():
    filecontent = unpersist(filecontent_path)
    return filecontent[["id", "user"]].rename({"id": "src", "user": "dst"}, axis=1)

One or several objectives could be used

In [11]:
class Trainer(SamplingMultitaskTrainer):
    def create_objectives(self, dataset, tokenizer_path):
        self.objectives = nn.ModuleList()
        
        self.objectives.append(
            self._create_subgraph_objective(
                objective_name="VariableMisuseClf",
                objective_class=SCAAClassifierObjective,
                dataset=dataset,
                tokenizer_path=tokenizer_path,
                labels_fn=load_labels,
            )
        )

In [12]:
# %tensorboard --logdir data
from tensorboard import program

tracking_address = './eamples/one_to_one/' # the path of your log file.

tb = program.TensorBoard()
tb.configure(argv=[None, '--logdir', tracking_address])
url = tb.launch()
print(f"Tensorflow listening on {url}")


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784



Tensorflow listening on http://localhost:6006/


In [ ]:
training_procedure(
    dataset, 
    model_name=RGGAN, 
    model_params=config["MODEL"],
    trainer_params=config["TRAINING"],
    model_base_path=get_model_base(config["TRAINING"], get_name(RGGAN, str(datetime.now()))),
    trainer=Trainer
)

Precompute Target Embeddings:   0%|                                                                                     | 0/44 [00:00<?, ?it/s]

Number of nodes 758247


Epoch 0:   0%|                                                                                                          | 0/42 [00:00<?, ?it/s]

embs torch.Size([128, 100])


Epoch 0:   2%|██▎                                                                                               | 1/42 [00:15<10:29, 15.35s/it]

embs torch.Size([128, 100])


Epoch 0:   5%|████▋                                                                                             | 2/42 [00:30<10:10, 15.26s/it]

embs torch.Size([128, 100])


Epoch 0:   7%|███████                                                                                           | 3/42 [00:44<09:44, 14.99s/it]

embs torch.Size([128, 100])
